# 🎓 ATLAS: Academic Task and Learning Agent System

## 학습 노트북 - LangGraph 멀티 에이전트 시스템

---

### 📚 이 노트북에서 배울 내용

1. **ReACT (Reasoning + Acting) 패턴**: AI 에이전트가 사고하고 행동하는 구조화된 방법
2. **멀티 에이전트 시스템**: 여러 전문 에이전트가 협업하는 시스템 설계
3. **LangGraph**: 복잡한 AI 워크플로우를 그래프로 표현하는 프레임워크
4. **비동기 프로그래밍**: asyncio를 활용한 효율적인 병렬 처리

---

### 🎯 ATLAS 시스템 개요

ATLAS는 학생들의 학업 관리를 돕는 **지능형 멀티 에이전트 시스템**입니다.

#### 핵심 에이전트 구성

```
┌─────────────────────┐
│  Coordinator Agent  │  ← 마스터 에이전트 (조율)
└──────────┬──────────┘
           │
     ┌─────┴─────┬─────────────┐
     │           │             │
┌────▼────┐ ┌───▼─────┐ ┌────▼──────┐
│ Planner │ │NoteWriter│ │  Advisor  │
│  Agent  │ │  Agent   │ │   Agent   │
└─────────┘ └──────────┘ └───────────┘
```

- **Coordinator**: 다른 에이전트들을 조율하고 작업 분배
- **Planner**: 일정 관리 및 학습 계획 수립
- **NoteWriter**: 학습 자료 생성 및 요약
- **Advisor**: 개인화된 학습 조언 제공

---

## 1️⃣ 필수 라이브러리 설치

### 왜 이 라이브러리들이 필요한가?

#### 🔧 핵심 라이브러리와 역할

| 라이브러리 | 역할 | 중요한 이유 |
|-----------|------|------------|
| **langgraph** | AI 워크플로우를 그래프로 표현 | 에이전트 간 데이터 흐름과 제어 흐름을 명확하게 관리 |
| **langchain** | LLM 통합 및 프롬프트 관리 | 구조화된 메시지 처리와 체인 구성 |
| **openai** | OpenAI 호환 API 호출 | NVIDIA NeMo를 OpenAI 인터페이스로 사용 |
| **pydantic** | 데이터 검증 및 타입 안정성 | 런타임 에러 방지 및 타입 체크 |
| **typing-extensions** | 고급 타입 힌팅 | TypedDict로 구조화된 딕셔너리 정의 |

---

In [ ]:
# 필수 패키지 설치
# Colab 환경이 아니라면 터미널에서 실행하세요

!pip install -q langgraph langchain langchain-core openai pydantic typing-extensions

## 2️⃣ 라이브러리 Import

### 각 라이브러리의 용도

```python
# 비동기 프로그래밍
import asyncio  # 여러 에이전트를 동시에 실행하기 위한 비동기 처리

# LangGraph
from langgraph.graph import StateGraph, END  # 상태 기반 워크플로우 그래프 구성

# LangChain  
from langchain_core.messages import HumanMessage, AIMessage  # 대화 구조화

# OpenAI (NVIDIA NeMo 호출용)
from openai import AsyncOpenAI  # 비동기 LLM API 호출

# Pydantic
from pydantic import BaseModel  # 데이터 모델 검증
```

---

In [ ]:
# ========================================
# 필수 라이브러리 Import
# ========================================

# === 비동기 프로그래밍 ===
# asyncio: 여러 에이전트를 동시에 실행하여 성능 최적화
import asyncio
from typing import List, Dict, Optional, Any

# === 날짜/시간 처리 ===
# 일정 관리에 필수적인 datetime 모듈
from datetime import datetime, timezone, timedelta

# === 데이터 처리 ===
# JSON 형식의 데이터 파싱 및 생성
import json

# === LangGraph: 워크플로우 구성 ===
# StateGraph: 상태를 공유하며 흐르는 그래프 구조
# END: 그래프의 종료 노드
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# === LangChain: LLM 메시지 처리 ===
# 사용자와 AI의 대화를 구조화
from langchain_core.messages import HumanMessage, AIMessage

# === OpenAI API ===
# NVIDIA NeMo 모델을 OpenAI 호환 인터페이스로 호출
from openai import AsyncOpenAI

# === Pydantic: 데이터 검증 ===
# BaseModel을 상속하여 타입 안정성 확보
from pydantic import BaseModel

print("✅ 모든 라이브러리 임포트 완료")

## 3️⃣ API 키 설정

### 🔑 NVIDIA API 키 발급 방법

1. [NVIDIA NGC](https://build.nvidia.com) 접속
2. 무료 계정 생성
3. API 키 발급
4. 아래 셀에 입력

### ⚠️ 보안 주의사항

- API 키는 절대 공개 저장소에 올리지 마세요
- 환경 변수나 별도 설정 파일 사용 권장
- Colab에서는 Secrets 기능 활용

---

In [ ]:
# ========================================
# API 키 설정
# ========================================

# 방법 1: 직접 입력 (학습용)
NVIDIA_API_KEY = "your-api-key-here"

# 방법 2: 환경 변수 사용 (권장)
# import os
# NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

# 방법 3: Google Colab Secrets
# from google.colab import userdata
# NVIDIA_API_KEY = userdata.get('NVIDIA_API_KEY')

if NVIDIA_API_KEY == "your-api-key-here":
    print("⚠️  위 셀에 실제 API 키를 입력해주세요!")
else:
    print("✅ API 키 설정 완료")

## 4️⃣ 데이터 모델 정의

### 📋 TypedDict vs 일반 Dict

#### 일반 Dict의 문제점
```python
state = {"messages": [], "profile": {}}
# 오타가 있어도 실행 시까지 모름
print(state["mesages"])  # KeyError!
```

#### TypedDict의 장점
```python
class State(TypedDict):
    messages: List[Any]
    profile: Dict

# IDE가 자동완성 제공
# 타입 체커가 오타 미리 발견
```

### 🎯 AcademicState 구조

- `messages`: 사용자-AI 대화 히스토리
- `profile`: 학생 프로필 (학습 스타일, 전공 등)
- `calendar`: 캘린더 이벤트 (시험, 과제 마감일)
- `tasks`: 할 일 목록
- `results`: 에이전트 실행 결과

---

In [ ]:
# ========================================
# 핵심 데이터 모델 정의
# ========================================

class AcademicState(TypedDict):
    """
    학업 관리 시스템의 전체 상태를 나타내는 타입
    
    이 상태는 LangGraph의 모든 노드(에이전트)를 거치며 전달됩니다.
    각 에이전트는 이 상태를 읽고 업데이트할 수 있습니다.
    
    TypedDict를 사용하는 이유:
    1. 타입 안정성: 잘못된 키 사용 방지
    2. IDE 지원: 자동완성 및 타입 힌트
    3. 문서화: 상태 구조를 명확히 표현
    """
    messages: List[Any]      # 대화 히스토리 (HumanMessage, AIMessage)
    profile: Dict            # 학생 프로필 (학습 선호도, 전공 등)
    calendar: Dict           # 캘린더 이벤트 (시험, 과제 마감일)
    tasks: Dict              # 할 일 목록
    results: Dict            # 에이전트 실행 결과 저장소


# ========================================
# Pydantic 모델: 강력한 데이터 검증
# ========================================

class AgentAction(BaseModel):
    """
    에이전트의 '행동'을 정의하는 모델
    
    ReACT 패턴의 'Action' 단계를 구조화합니다.
    Pydantic이 자동으로 타입 검증 및 변환을 수행합니다.
    
    사용 예시:
        action = AgentAction(
            action="search_calendar",
            thought="시험 일정을 확인해야 함",
            tool="calendar_search"
        )
    """
    action: str                          # 행동 이름 (예: "search_calendar")
    thought: str                         # 행동의 이유 (ReACT의 사고 과정)
    tool: Optional[str] = None           # 사용할 도구 (선택사항)
    action_input: Optional[Dict] = None  # 행동에 필요한 파라미터


class AgentOutput(BaseModel):
    """
    에이전트의 '관찰 결과'를 정의하는 모델
    
    ReACT 패턴의 'Observation' 단계를 구조화합니다.
    
    사용 예시:
        output = AgentOutput(
            observation="3개의 예정된 이벤트를 찾았습니다",
            output={"events": [...], "count": 3}
        )
    """
    observation: str    # 관찰한 내용 (자연어)
    output: Dict        # 구조화된 출력 데이터

print("✅ 데이터 모델 정의 완료")

## 5️⃣ LLM 설정: NVIDIA NeMo

### 🤖 왜 NVIDIA NeMo인가?

1. **고성능**: 340B 파라미터의 대규모 모델
2. **무료 API**: 일정량까지 무료 사용 가능
3. **OpenAI 호환**: 익숙한 인터페이스
4. **비동기 지원**: 병렬 처리에 최적화

### 📊 주요 파라미터 설명

- **temperature**: 응답의 창의성 조절
  - 0.0: 매우 결정적 (같은 입력 → 같은 출력)
  - 0.5: 균형잡힌 (기본값)
  - 1.0: 매우 창의적 (다양한 응답)

- **max_tokens**: 응답 최대 길이
  - 짧은 답변: 256
  - 중간 길이: 1024 (기본값)
  - 긴 답변: 2048+

---

In [ ]:
# ========================================
# LLM 설정 클래스
# ========================================

class LLMConfig:
    """
    LLM(대형 언어 모델)의 설정을 중앙에서 관리
    
    클래스 변수를 사용하여 모든 인스턴스가 동일한 설정을 공유합니다.
    필요시 인스턴스별로 다른 설정도 가능합니다.
    """
    base_url: str = "https://integrate.api.nvidia.com/v1"  # NVIDIA API 엔드포인트
    model: str = "nvidia/nemotron-4-340b-instruct"         # 모델 이름
    max_tokens: int = 1024                                 # 최대 응답 길이
    default_temp: float = 0.5                              # 기본 temperature


# ========================================
# NeMo LLM 래퍼 클래스
# ========================================

class NeMoLLaMa:
    """
    NVIDIA NeMo 모델과 통신하기 위한 래퍼 클래스
    
    OpenAI의 AsyncOpenAI 클라이언트를 사용하여 NVIDIA API를 호출합니다.
    (NVIDIA API가 OpenAI 호환 인터페이스를 제공하기 때문)
    
    주요 기능:
    - 비동기 텍스트 생성 (agenerate)
    - API 인증 확인 (check_auth)
    - 자동 에러 핸들링
    
    비동기를 사용하는 이유:
    여러 에이전트가 동시에 LLM을 호출할 때 블로킹 없이
    효율적으로 처리할 수 있습니다.
    """
    
    def __init__(self, api_key: str):
        """LLM 클라이언트 초기화"""
        self.config = LLMConfig()
        self.client = AsyncOpenAI(
            base_url=self.config.base_url,
            api_key=api_key
        )
        self._is_authenticated = False
    
    async def check_auth(self) -> bool:
        """
        API 키가 유효한지 확인
        
        간단한 테스트 메시지를 보내서 인증 상태를 확인합니다.
        실제 프로덕션에서는 시작 시 한 번만 호출하면 됩니다.
        """
        test_message = [{"role": "user", "content": "test"}]
        try:
            await self.agenerate(test_message, temperature=0.1)
            self._is_authenticated = True
            print("✅ API 인증 성공")
            return True
        except Exception as e:
            print(f"❌ API 인증 실패: {str(e)}")
            return False
    
    async def agenerate(
        self,
        messages: List[Dict],
        temperature: Optional[float] = None
    ) -> str:
        """
        비동기 텍스트 생성 - 가장 중요한 메서드
        
        모든 에이전트가 이 메서드를 통해 LLM을 호출합니다.
        'async/await'를 사용하여 비동기적으로 실행됩니다.
        
        Args:
            messages: 대화 메시지 리스트
                형식: [{"role": "user", "content": "질문"}, ...]
            temperature: 응답의 창의성 (0.0~1.0)
        
        Returns:
            LLM의 응답 텍스트
        
        사용 예시:
            messages = [
                {"role": "system", "content": "당신은 학습 도우미입니다"},
                {"role": "user", "content": "시험 준비 방법은?"}
            ]
            response = await llm.agenerate(messages, temperature=0.7)
        """
        completion = await self.client.chat.completions.create(
            model=self.config.model,
            messages=messages,
            temperature=temperature or self.config.default_temp,
            max_tokens=self.config.max_tokens,
            stream=False  # 전체 응답을 한 번에 받음
        )
        return completion.choices[0].message.content

print("✅ LLM 클래스 정의 완료")

## 6️⃣ LLM 초기화 및 테스트

### 체크포인트

이 단계에서 확인할 사항:
- ✅ LLM 인스턴스 생성 성공
- ✅ API 키 유효성 확인
- ✅ 간단한 테스트 응답 확인

---

In [ ]:
# ========================================
# LLM 인스턴스 생성
# ========================================

llm = NeMoLLaMa(NVIDIA_API_KEY)

print("✅ LLM 인스턴스 생성 완료")
print(f"📊 모델: {llm.config.model}")
print(f"📏 최대 토큰: {llm.config.max_tokens}")
print(f"🌡️  기본 Temperature: {llm.config.default_temp}")

In [ ]:
# ========================================
# API 인증 테스트
# ========================================

auth_result = await llm.check_auth()

if auth_result:
    print("\n🎉 모든 시스템 정상 작동!")
    print("이제 에이전트를 구축할 준비가 되었습니다.")
else:
    print("\n⚠️  API 키를 다시 확인해주세요.")

## 7️⃣ 샘플 데이터 준비

### 실습용 데이터

실제 환경에서는:
- Google Calendar API에서 일정 가져오기
- Notion API에서 할 일 가져오기
- 학교 시스템에서 프로필 가져오기

학습을 위해 샘플 데이터를 직접 정의합니다.

---

In [ ]:
# ========================================
# 샘플 학생 프로필
# ========================================

sample_profile = {
    "id": "student_001",
    "personal_info": {
        "name": "김철수",
        "major": "컴퓨터공학",
        "academic_year": 3  # 3학년
    },
    "learning_preferences": {
        "learning_style": {
            "primary": "visual",      # 시각적 학습 선호
            "secondary": "hands-on"   # 실습 선호
        },
        "study_patterns": {
            "best_time": "morning",   # 아침에 집중력이 가장 좋음
            "session_length": 90,     # 90분 학습 선호
            "break_length": 15        # 15분 휴식
        }
    },
    "academic_info": {
        "current_courses": [
            {
                "name": "데이터구조",
                "difficulty": "medium",
                "credits": 3
            },
            {
                "name": "알고리즘",
                "difficulty": "hard",
                "credits": 3
            },
            {
                "name": "운영체제",
                "difficulty": "hard",
                "credits": 3
            }
        ]
    }
}

# ========================================
# 샘플 캘린더 데이터
# ========================================

sample_calendar = {
    "events": [
        {
            "summary": "알고리즘 중간고사",
            "start": {"dateTime": "2025-11-01T09:00:00Z"},
            "end": {"dateTime": "2025-11-01T11:00:00Z"},
            "description": "3장~5장 범위"
        },
        {
            "summary": "데이터구조 과제 제출",
            "start": {"dateTime": "2025-10-28T23:59:00Z"},
            "end": {"dateTime": "2025-10-28T23:59:00Z"},
            "description": "이진 탐색 트리 구현"
        },
        {
            "summary": "운영체제 팀 프로젝트 발표",
            "start": {"dateTime": "2025-11-05T14:00:00Z"},
            "end": {"dateTime": "2025-11-05T16:00:00Z"},
            "description": "스케줄링 알고리즘 분석"
        }
    ]
}

# ========================================
# 샘플 할 일 목록
# ========================================

sample_tasks = {
    "tasks": [
        {
            "title": "알고리즘 3장 복습",
            "status": "needsAction",
            "due": "2025-10-25T18:00:00Z",
            "priority": "high"
        },
        {
            "title": "데이터구조 과제 코딩",
            "status": "needsAction",
            "due": "2025-10-27T18:00:00Z",
            "priority": "high"
        },
        {
            "title": "운영체제 PPT 준비",
            "status": "needsAction",
            "due": "2025-11-03T18:00:00Z",
            "priority": "medium"
        }
    ]
}

print("✅ 샘플 데이터 준비 완료\n")
print(f"👤 학생: {sample_profile['personal_info']['name']}")
print(f"📚 전공: {sample_profile['personal_info']['major']}")
print(f"📖 수강 과목: {len(sample_profile['academic_info']['current_courses'])}개")
print(f"📅 예정된 이벤트: {len(sample_calendar['events'])}개")
print(f"✅ 할 일: {len(sample_tasks['tasks'])}개")

## 8️⃣ 실습 1: 간단한 LLM 호출

### 학습 목표

1. 비동기 함수 작성법 (`async def`)
2. 비동기 함수 호출법 (`await`)
3. 프롬프트 구조 이해 (system, user 역할)
4. temperature 파라미터의 효과

---

In [ ]:
# ========================================
# 실습 1: 기본 LLM 호출
# ========================================

async def simple_study_advice():
    """
    간단한 학습 조언을 받는 함수
    
    포인트:
    1. system 메시지: AI의 역할과 성격 정의
    2. user 메시지: 실제 질문이나 요청
    3. temperature: 응답의 다양성 조절
    """
    messages = [
        {
            "role": "system",
            "content": "당신은 대학생들의 학습을 돕는 친절한 AI 멘토입니다. 실용적이고 구체적인 조언을 제공합니다."
        },
        {
            "role": "user",
            "content": "알고리즘 시험이 일주일 남았는데, 효과적으로 준비하는 방법을 알려주세요."
        }
    ]
    
    # LLM 호출 (비동기)
    response = await llm.agenerate(messages, temperature=0.7)
    return response


# 실행
print("🤔 질문: 알고리즘 시험이 일주일 남았는데, 효과적으로 준비하는 방법은?\n")
advice = await simple_study_advice()
print("🤖 AI 멘토의 조언:")
print(advice)

## 9️⃣ 실습 2: Temperature 비교

### Temperature의 효과 체험

같은 질문에 대해 다른 temperature로 호출하여 차이를 확인합니다.

---

In [ ]:
# ========================================
# 실습 2: Temperature 효과 비교
# ========================================

async def compare_temperatures():
    """
    동일한 질문에 대해 다른 temperature로 응답을 비교
    """
    question = "데이터구조를 공부하는 가장 좋은 방법은?"
    
    messages = [
        {"role": "system", "content": "당신은 학습 전문가입니다."},
        {"role": "user", "content": question}
    ]
    
    print(f"📝 질문: {question}\n")
    print("=" * 60)
    
    # Temperature 0.1 (매우 일관적)
    print("\n🌡️  Temperature = 0.1 (일관적/결정적)")
    print("-" * 60)
    response_low = await llm.agenerate(messages, temperature=0.1)
    print(response_low[:200] + "...\n")  # 처음 200자만 출력
    
    # Temperature 0.9 (매우 창의적)
    print("\n🌡️  Temperature = 0.9 (창의적/다양함)")
    print("-" * 60)
    response_high = await llm.agenerate(messages, temperature=0.9)
    print(response_high[:200] + "...\n")  # 처음 200자만 출력

# 실행
await compare_temperatures()

## 🔟 실습 3: 학생 프로필 기반 맞춤 조언

### 컨텍스트를 활용한 개인화

학생의 프로필 정보를 프롬프트에 포함시켜
더 개인화된 조언을 받습니다.

---

In [ ]:
# ========================================
# 실습 3: 프로필 기반 개인화 조언
# ========================================

async def personalized_advice():
    """
    학생 프로필을 고려한 맞춤형 학습 조언
    
    중요: 프롬프트에 학생의 학습 스타일, 선호 시간대 등을
    포함시켜 더 실용적인 조언을 얻습니다.
    """
    # 학생 정보 추출
    student = sample_profile['personal_info']
    preferences = sample_profile['learning_preferences']
    
    # 컨텍스트가 풍부한 프롬프트 작성
    context = f"""
학생 정보:
- 이름: {student['name']}
- 전공: {student['major']}
- 학습 스타일: {preferences['learning_style']['primary']} (시각적 학습 선호)
- 최적 학습 시간: {preferences['study_patterns']['best_time']} (아침)
- 선호 학습 시간: {preferences['study_patterns']['session_length']}분

현재 상황:
- 알고리즘 중간고사 1주일 후
- 데이터구조 과제 3일 후 마감
"""
    
    messages = [
        {
            "role": "system",
            "content": "당신은 학생의 개인적 특성을 고려하여 최적화된 학습 계획을 제안하는 AI 멘토입니다."
        },
        {
            "role": "user",
            "content": f"{context}\n\n이 학생을 위한 이번 주 학습 계획을 구체적으로 제안해주세요."
        }
    ]
    
    response = await llm.agenerate(messages, temperature=0.6)
    return response


# 실행
print("🎯 개인화된 학습 계획\n")
plan = await personalized_advice()
print(plan)

## 🎓 학습 정리

### ✅ 지금까지 배운 내용

1. **라이브러리 설정**
   - LangGraph, LangChain, OpenAI의 역할 이해
   - 각 라이브러리가 필요한 이유

2. **데이터 모델링**
   - TypedDict로 타입 안정성 확보
   - Pydantic으로 데이터 검증

3. **LLM 사용법**
   - 비동기 프로그래밍 (async/await)
   - 프롬프트 구조 (system, user)
   - Temperature 파라미터 조절

4. **개인화 전략**
   - 컨텍스트를 활용한 맞춤형 응답
   - 학생 프로필 기반 조언

---

### 🚀 다음 단계

이 기초를 바탕으로:
1. **ReACT Agent 패턴** 구현
2. **멀티 에이전트 시스템** 구축
3. **LangGraph로 워크플로우** 연결
4. **실제 API 통합** (Google Calendar, Notion 등)

---

### 📚 참고 자료

- [LangGraph 공식 문서](https://langchain-ai.github.io/langgraph/)
- [LangChain 문서](https://python.langchain.com/)
- [NVIDIA NeMo](https://build.nvidia.com)
- [ReACT 논문](https://arxiv.org/abs/2210.03629)
- [Pydantic 문서](https://docs.pydantic.dev/)

---

### 💡 추가 실습 아이디어

1. 여러 질문을 동시에 처리하는 비동기 함수 작성
2. 대화 히스토리를 유지하는 채팅봇 만들기
3. 학생별 프로필을 JSON 파일로 저장/불러오기
4. 다양한 temperature로 응답 품질 비교

---

## 🎉 축하합니다!

LangGraph 멀티 에이전트 시스템의 기초를 완성했습니다.

이제 다음 노트북에서 본격적인 에이전트 구현으로 넘어갑니다.

---